<a href="https://colab.research.google.com/github/nikhilcoding0013/Basketball-Game-Predictor/blob/main/1A_WhartonDataSciComp_Final_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
df =pd.read_csv("games_2022 - games_2022 (1).csv")
df1= pd.read_csv('games_2022 - games_2022 (1).csv')
df_games = df1.merge(df, on="game_id", suffixes=("_A", "_B"))
df_games = df_games[df_games["team_A"] != df_games["team_B"]]
columns_needed = [
    "team_A", "team_B", "team_score_A", "team_score_B",
    "FGA_3_A", "FGM_3_A", "FGA_2_A", "FGM_2_A", "FTA_A", "FTM_A",
    "AST_A", "BLK_A", "STL_A", "TOV_A", "DREB_A", "OREB_A",
    "FGA_3_B", "FGM_3_B", "FGA_2_B", "FGM_2_B", "FTA_B", "FTM_B",
    "AST_B", "BLK_B", "STL_B", "TOV_B", "DREB_B", "OREB_B"
]
df_games = df_games[columns_needed].drop_duplicates()
stat_list = ['AST', 'BLK', 'STL', 'TOV', 'DREB', 'OREB']
for stat in stat_list:
    df_games[f'{stat}_diff'] = df_games[f'{stat}_A'] - df_games[f'{stat}_B']
df_games['result'] = (df_games['team_score_A'] > df_games['team_score_B']).astype(int)
X = df_games[[f'{stat}_diff' for stat in stat_list]]
y = df_games['result']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
learned_coeffs = model.coef_[0]
predicted_stat_weighting = dict(zip(stat_list, learned_coeffs))
print(predicted_stat_weighting)

{'AST': 1.2096897813756242, 'BLK': 0.11042661188571065, 'STL': 0.1753022616798368, 'TOV': -4.651538446135684, 'DREB': 5.467526587946814, 'OREB': -0.16406959836102708}


In [ ]:
#ELO model

import pandas as pd
import numpy as np
# Initialize Elo ratings dictionary
elo_ratings = {}  # This will store each team's current Elo rating

def get_elo(team):
    """Retrieve the team's current Elo rating. If they don't have one, assign default 1500."""
    if team not in elo_ratings:
        elo_ratings[team] = 1500  # Default starting Elo
    return elo_ratings[team]

#logistic function which calculates exprected win prob for team_A
def expected_win_prob(team_A, team_B):
    """Calculate the probability of Team A winning against Team B based on Elo ratings."""
    R_A = get_elo(team_A)
    R_B = get_elo(team_B)
    diff=R_A-R_B
    return 1 / (1 + np.exp(-diff / 800))

#all this stuff below is to adjust the elo rating. look at elo update functions: it's based off what the expected win percentage was

def update_elo_with_advanced_stats(team_A, team_B, score_A, score_B,
                                   FGM_3_A, FGA_3_A, FGM_2_A, FGA_2_A, FTM_A, FTA_A, AST_A, BLK_A, STL_A, TOV_A, DREB_A, OREB_A,
                                   FGM_3_B, FGA_3_B, FGM_2_B, FGA_2_B, FTM_B, FTA_B, AST_B, BLK_B, STL_B, TOV_B, DREB_B, OREB_B):
    """Update Elo ratings with shooting efficiency and advanced stats like AST, BLK, STL, TOV, REB."""

    K = 40 #adjust based on rate of change for elo

    P_A = expected_win_prob(team_A, team_B)  # Expected probability of Team A winning
    P_B = 1 - P_A  # Expected probability of Team B winning

    # Determine actual result
    S_A, S_B = (1, 0) if score_A > score_B else (0, 1)  # 1 if win, 0 if loss

    # Calculate shooting efficiencies
    eff_3_A = FGM_3_A / FGA_3_A if FGA_3_A > 0 else 0  # 3PT efficiency
    eff_2_A = FGM_2_A / FGA_2_A if FGA_2_A > 0 else 0  # 2PT efficiency
    eff_ft_A = FTM_A / FTA_A if FTA_A > 0 else 0       # Free throw efficiency

    eff_3_B = FGM_3_B / FGA_3_B if FGA_3_B > 0 else 0
    eff_2_B = FGM_2_B / FGA_2_B if FGA_2_B > 0 else 0
    eff_ft_B = FTM_B / FTA_B if FTA_B > 0 else 0

    # Performance-based adjustments
    performance_factor = 10

    shooting_adjustment_A = performance_factor * (
        (eff_3_A - 0.35) + (eff_2_A - 0.50) + (eff_ft_A - 0.75)
    )
    shooting_adjustment_B = performance_factor * (
        (eff_3_B - 0.35) + (eff_2_B - 0.50) + (eff_ft_B - 0.75)
    )

    # Advanced stat impact
    stat_weighting = predicted_stat_weighting

    advanced_stat_adjustment_A = (
        AST_A * stat_weighting["AST"] +
        BLK_A * stat_weighting["BLK"] +
        STL_A * stat_weighting["STL"] +
        TOV_A * stat_weighting["TOV"] +
        DREB_A * stat_weighting["DREB"] +
        OREB_A * stat_weighting["OREB"]
    )

    advanced_stat_adjustment_B = (
        AST_B * stat_weighting["AST"] +
        BLK_B * stat_weighting["BLK"] +
        STL_B * stat_weighting["STL"] +
        TOV_B * stat_weighting["TOV"] +
        DREB_B * stat_weighting["DREB"] +
        OREB_B * stat_weighting["OREB"]
    )

    # Final Elo update
    elo_ratings[team_A] = get_elo(team_A) + K * (S_A - P_A) + shooting_adjustment_A + advanced_stat_adjustment_A
    elo_ratings[team_B] = get_elo(team_B) + K * (S_B - P_B) + shooting_adjustment_B + advanced_stat_adjustment_B
    return elo_ratings[team_A], elo_ratings[team_B]
final_elo_ratings = {}
df2 = df.drop(columns=['game_date', 'F_tech', 'F_personal', 'largest_lead', 'OT_length_min_tot', 'attendance', 'tz_dif_H_E', 'prev_game_dist', 'rest_days', 'home_away', 'home_away_NS',
       'travel_dist', 'TOV_team'])

df2.columns
print(df2.head())
# Loop through every two rows (each game has two rows)
for i in range(0, len(df), 2):
    team_A = df2.iloc[i]  # First row (Team 1)
    team_B = df2.iloc[i+1]  # Second row (Team 2)

    # Call the Elo update function
    new_elo_A, new_elo_B = update_elo_with_advanced_stats(
        team_A['team'], team_B['team'], team_A['team_score'], team_B['team_score'],
        team_A['FGM_3'], team_A['FGA_3'], team_A['FGM_2'], team_A['FGA_2'], team_A['FTM'], team_A['FTA'],
        team_A['AST'], team_A['BLK'], team_A['STL'], team_A['TOV'], team_A['DREB'], team_A['OREB'],
        team_B['FGM_3'], team_B['FGA_3'], team_B['FGM_2'], team_B['FGA_2'], team_B['FTM'], team_B['FTA'],
        team_B['AST'], team_B['BLK'], team_B['STL'], team_B['TOV'], team_B['DREB'], team_B['OREB']
        )
    final_elo_ratings[team_A['team']] = new_elo_A
    final_elo_ratings[team_B['team']] = new_elo_B

# Convert to DataFrame for better visualization
final_elo_df = pd.DataFrame(list(final_elo_ratings.items()), columns=['Team', 'Final_Elo'])
final_elo_df = final_elo_df.sort_values(by='Final_Elo', ascending=False)
# Display final Elo ratings
print(final_elo_df)


          game_id                       team  FGA_2  FGM_2  FGA_3  FGM_3  FTA  \
0  game_2022_2011      georgia_lady_bulldogs     50     22     11      5    6   
1  game_2022_2011                 lsu_tigers     50     24     11      4   15   
2  game_2022_2012            missouri_tigers     43     18     15      7   16   
3  game_2022_2012   south_carolina_gamecocks     55     23     21      6    9   
4  game_2022_2013  tennessee_lady_volunteers     41     20     15      4   15   

   FTM  AST  BLK  STL  TOV  DREB  OREB  team_score  opponent_team_score  \
0    3   14    7    7   18    25    11          62                   68   
1    8   15    2   15   14    25    11          68                   62   
2   13   10    1    4    8    31     6          70                   69   
3    5   15    8    3    8    27    20          69                   70   
4   10   16    8    5   15    34    12          62                   44   

   notD1_incomplete  
0             False  
1             Fals

In [ ]:
regions_df = pd.read_csv("Team Region Groups - Sheet1.csv")

# Merge final Elo ratings with region data
merged_df = final_elo_df.merge(regions_df, left_on="Team", right_on="team")

# Rank teams within each region based on Final_Elo
merged_df["Region_Rank"] = merged_df.groupby("region")["Final_Elo"].rank(ascending=False, method='first')

# Filter top 16 teams per region
top_16_per_region = merged_df[merged_df["Region_Rank"] <= 16]

# Sort by region and rank
top_16_per_region = top_16_per_region.sort_values(by=["region", "Region_Rank"])

# Print rankings
for region, group in top_16_per_region.groupby("region"):
    print(f"Top 16 Teams in {region}:")
    print(group[["Region_Rank", "Team", "Final_Elo"]].to_string(index=False))
    print("\n")


Top 16 Teams in North:
 Region_Rank                          Team   Final_Elo
         1.0      south_carolina_gamecocks 5367.865246
         2.0                  troy_trojans 5015.398231
         3.0     florida_gulf_coast_eagles 5002.997431
         4.0                    lsu_tigers 4891.309784
         5.0     tennessee_lady_volunteers 4864.677615
         6.0   georgia_tech_yellow_jackets 4764.948168
         7.0           arkansas_razorbacks 4733.212810
         8.0     jackson_state_lady_tigers 4687.426837
         9.0             tulane_green_wave 4614.435617
        10.0           south_florida_bulls 4587.603052
        11.0  jacksonville_state_gamecocks 4500.285244
        12.0               charlotte_49ers 4499.573300
        13.0 middle_tennessee_blue_raiders 4477.945509
        14.0           high_point_panthers 4448.581204
        15.0                belmont_bruins 4287.933689
        16.0 louisiana_tech_lady_techsters 4255.863631


Top 16 Teams in South:
 Region_Rank     